In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.getOrCreate()

C:\Users\RAVI\anaconda3\envs\vizuamatix\lib\site-packages\pyspark\context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [3]:
df=spark.read.csv('E:\Desktop\Datasets\Loan payments data.csv',header=True)

In [4]:
df.show()

+-----------+-----------+---------+-----+--------------+---------+---------------+-------------+---+--------------------+------+
|    Loan_ID|loan_status|Principal|terms|effective_date| due_date|  paid_off_time|past_due_days|age|           education|Gender|
+-----------+-----------+---------+-----+--------------+---------+---------------+-------------+---+--------------------+------+
|xqd20166231|    PAIDOFF|     1000|   30|      9/8/2016|10/7/2016|9/14/2016 19:31|         null| 45|High School or Below|  male|
|xqd20168902|    PAIDOFF|     1000|   30|      9/8/2016|10/7/2016| 10/7/2016 9:00|         null| 50|            Bechalor|female|
|xqd20160003|    PAIDOFF|     1000|   30|      9/8/2016|10/7/2016|9/25/2016 16:58|         null| 33|            Bechalor|female|
|xqd20160004|    PAIDOFF|     1000|   15|      9/8/2016|9/22/2016|9/22/2016 20:00|         null| 27|             college|  male|
|xqd20160005|    PAIDOFF|     1000|   30|      9/9/2016|10/8/2016|9/23/2016 21:36|         null| 

In [5]:
#convert age datatype to integer.....
df=df.withColumn("age",df.age.cast('int'))

In [6]:
#convert Principal datatype to integer.....
df=df.withColumn("Principal",df.Principal.cast('int'))

In [7]:
#convert Principal datatype to integer.....
df=df.withColumn("terms",df.terms.cast('int'))

In [8]:
df.dtypes

[('Loan_ID', 'string'),
 ('loan_status', 'string'),
 ('Principal', 'int'),
 ('terms', 'int'),
 ('effective_date', 'string'),
 ('due_date', 'string'),
 ('paid_off_time', 'string'),
 ('past_due_days', 'string'),
 ('age', 'int'),
 ('education', 'string'),
 ('Gender', 'string')]

In [9]:
df1=df.groupBy('education').count()

In [10]:
df1.show()

+--------------------+-----+
|           education|count|
+--------------------+-----+
|             college|  220|
|     Master or Above|    4|
|            Bechalor|   67|
|High School or Below|  209|
+--------------------+-----+



In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col

In [12]:
#get the first element of each education groups(each group order by Loan_ID and get the 1st value)
windowSpec  = Window.partitionBy('education').orderBy("Loan_ID")
df2=df.withColumn("Row_number",row_number().over(windowSpec)).filter(col("Row_number")==1)

In [13]:
df2.columns

['Loan_ID',
 'loan_status',
 'Principal',
 'terms',
 'effective_date',
 'due_date',
 'paid_off_time',
 'past_due_days',
 'age',
 'education',
 'Gender',
 'Row_number']

In [14]:
#select 'Loan_ID','loan_status','terms','age','education','Gender' columns in each group...........
df2.select('Loan_ID','loan_status','terms','age','education','Gender').show()

+-----------+------------------+-----+---+--------------------+------+
|    Loan_ID|       loan_status|terms|age|           education|Gender|
+-----------+------------------+-----+---+--------------------+------+
|xqd12160159|           PAIDOFF|   15| 28|             college|female|
|xqd20160060|           PAIDOFF|   15| 39|     Master or Above|  male|
|xqd20160003|           PAIDOFF|   30| 33|            Bechalor|female|
|xqd20110409|COLLECTION_PAIDOFF|   30| 28|High School or Below|  male|
+-----------+------------------+-----+---+--------------------+------+



In [15]:
#get the first element of each Gender groups(each group order by Loan_ID and get the 1st value)
windowSpec  = Window.partitionBy('Gender').orderBy("Loan_ID")
df3=df.withColumn("Row_number",row_number().over(windowSpec)).filter(col("Row_number")==1)

In [16]:
#select 'Loan_ID','loan_status','terms','age','education','Gender' columns in each group...........
df3.select('Loan_ID','loan_status','terms','age','education','Gender').show()

+-----------+------------------+-----+---+--------------------+------+
|    Loan_ID|       loan_status|terms|age|           education|Gender|
+-----------+------------------+-----+---+--------------------+------+
|xqd12160159|           PAIDOFF|   15| 28|             college|female|
|xqd20110409|COLLECTION_PAIDOFF|   30| 28|High School or Below|  male|
+-----------+------------------+-----+---+--------------------+------+



In [17]:
#get the first element of each age groups(each group order by Loan_ID and get the 1st value)
windowSpec  = Window.partitionBy('age').orderBy("Loan_ID")
df4=df.withColumn("Row_number",row_number().over(windowSpec)).filter(col("Row_number")==1)

In [18]:
#select 'Loan_ID','loan_status','terms','age','education','Gender' columns in each group...........
df4.select('Loan_ID','loan_status','terms','age','education','Gender').show()

+-----------+------------------+-----+---+--------------------+------+
|    Loan_ID|       loan_status|terms|age|           education|Gender|
+-----------+------------------+-----+---+--------------------+------+
|xqd20160029|           PAIDOFF|   30| 31|             college|  male|
|xqd20160054|           PAIDOFF|   30| 34|High School or Below|  male|
|xqd12160159|           PAIDOFF|   15| 28|             college|female|
|xqd20160010|           PAIDOFF|   15| 26|             college|  male|
|xqd20160004|           PAIDOFF|   15| 27|             college|  male|
|xqd20160059|           PAIDOFF|   15| 44|High School or Below|female|
|xqd20160047|           PAIDOFF|   30| 22|High School or Below|  male|
|xqd20160139|           PAIDOFF|   15| 47|High School or Below|  male|
|xqd20160123|           PAIDOFF|   15| 20|             college|  male|
|xqd20160017|           PAIDOFF|   30| 40|High School or Below|  male|
|xqd20160246|           PAIDOFF|   30| 19|High School or Below|female|
|xqd20

In [19]:
#get the first element of each loan_status groups(each group order by Loan_ID and get the 1st value)
windowSpec  = Window.partitionBy('loan_status').orderBy("Loan_ID")
df5=df.withColumn("Row_number",row_number().over(windowSpec)).filter(col("Row_number")==1)

In [20]:
#select 'Loan_ID','loan_status','terms','age','education','Gender' columns in each group...........
df5.select('Loan_ID','loan_status','terms','age','education','Gender').show()

+-----------+------------------+-----+---+--------------------+------+
|    Loan_ID|       loan_status|terms|age|           education|Gender|
+-----------+------------------+-----+---+--------------------+------+
|xqd20125284|        COLLECTION|   15| 35|High School or Below|  male|
|xqd20110409|COLLECTION_PAIDOFF|   30| 28|High School or Below|  male|
|xqd12160159|           PAIDOFF|   15| 28|             college|female|
+-----------+------------------+-----+---+--------------------+------+



In [21]:
#get the first element of each terms groups(each group order by Loan_ID and get the 1st value)
windowSpec  = Window.partitionBy('terms').orderBy("Loan_ID")
df6=df.withColumn("Row_number",row_number().over(windowSpec)).filter(col("Row_number")==1)

In [22]:
#select 'Loan_ID','loan_status','terms','age','education','Gender' columns in each group...........
df6.select('Loan_ID','loan_status','terms','age','education','Gender').show()

+-----------+------------------+-----+---+--------------------+------+
|    Loan_ID|       loan_status|terms|age|           education|Gender|
+-----------+------------------+-----+---+--------------------+------+
|xqd12160159|           PAIDOFF|   15| 28|             college|female|
|xqd20160011|           PAIDOFF|    7| 29|             college|  male|
|xqd20110409|COLLECTION_PAIDOFF|   30| 28|High School or Below|  male|
+-----------+------------------+-----+---+--------------------+------+



# Alternative way......................

In [23]:
#Here I got the max age value in each education group....(it's equal to when each group order by age descending order and get the 1st value)
df=df.groupby('education').max('age').show()

+--------------------+--------+
|           education|max(age)|
+--------------------+--------+
|             college|      51|
|     Master or Above|      50|
|            Bechalor|      50|
|High School or Below|      50|
+--------------------+--------+

